In [1]:
import pandas as pd
import os
import glob
import numpy as np
from datetime import datetime, timedelta
import re
import warnings

## Blood culture filter & calculate windows period

In [ ]:
# 设置文件夹路径
for year in range(2000, 2026):
    folder_path = f"./presumed_infection/2.Data/Blood_culture/{year}/"

    # 获取所有CSV文件
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

    if not csv_files:
        print("未找到CSV文件")
    else:
        print(f"找到 {len(csv_files)} 个CSV文件")

        all_data = []

        for csv_file in csv_files:
            try:
                # 读取文件
                df = pd.read_csv(csv_file, low_memory=False)
                file_name = os.path.basename(csv_file)

                # 查找筛选列
                filter_col = None
                for col in df.columns:
                    if "Bacterial Culture and ST Within Hospital Stay - Culture Test" in str(col):
                        filter_col = col
                        break

                if filter_col:
                    # 筛选包含Blood的行
                    blood_mask = df[filter_col].astype(str).str.contains("Blood", case=False, na=False)
                    filtered = df.loc[blood_mask]

                    if len(filtered) > 0:
                        # 只保留需要的列
                        columns_to_keep = [
                            "Reference Key",
                            "Institution (IPAS)",
                            "HN Number",
                            "Bacterial Culture and ST Within Hospital Stay - Ref. Date (yyyy-mm-dd HH:MM)"
                        ]

                        # 提取存在的列
                        existing_cols = [col for col in columns_to_keep if col in filtered.columns]

                        if existing_cols:
                            extracted_data = filtered[existing_cols].copy()
                            all_data.append(extracted_data)
                        else:
                            print(f"✗ {file_name}: 缺少必需列")
                    else:
                        print(f"  {file_name}: 无匹配行")
                else:
                    print(f"  {file_name}: 无筛选列")

            except Exception as e:
                print(f"✗ {os.path.basename(csv_file)} 错误: {e}")

        if all_data:
            # 合并数据
            combined = pd.concat(all_data, ignore_index=True)
            # 去除重复项
            before = len(combined)
            combined = combined.drop_duplicates()
            after = len(combined)

            # 处理日期和计算窗口期
            date_col = "Bacterial Culture and ST Within Hospital Stay - Ref. Date (yyyy-mm-dd HH:MM)"

            if date_col in combined.columns:
                # 将日期列转换为datetime类型
                combined[date_col] = pd.to_datetime(combined[date_col], errors='coerce')

                # 只保留年月日（去掉时间部分）
                combined[date_col] = combined[date_col].dt.date

                # 计算窗口期（前后两天）
                combined['window_period_start'] = combined[date_col] - timedelta(days=2)
                combined['window_period_end'] = combined[date_col] + timedelta(days=2)

                # 将所有日期列转换为字符串格式（只保留年月日）
                combined = combined.rename(columns={date_col: 'blood culture date'})
                combined['blood culture date'] = combined['blood culture date'].astype(str)
                combined['window_period_start'] = combined['window_period_start'].astype(str)
                combined['window_period_end'] = combined['window_period_end'].astype(str)

            # 保存结果
            output_path = f"./presumed_infection/2.Data/Window_Period/"
            os.makedirs(output_path, exist_ok=True)
            output_path = os.path.join(output_path, f"bloodculture_filter{year}.csv")
            combined.to_csv(output_path, index=False, encoding='utf-8-sig')

找到 14 个CSV文件
找到 15 个CSV文件
找到 14 个CSV文件
找到 14 个CSV文件
找到 16 个CSV文件
找到 14 个CSV文件
找到 11 个CSV文件
找到 12 个CSV文件
找到 13 个CSV文件
找到 7 个CSV文件
未找到CSV文件
未找到CSV文件
找到 10 个CSV文件
找到 13 个CSV文件
找到 15 个CSV文件
找到 14 个CSV文件
找到 15 个CSV文件
找到 34 个CSV文件
找到 52 个CSV文件
未找到CSV文件
找到 6 个CSV文件
找到 28 个CSV文件
找到 39 个CSV文件
找到 26 个CSV文件
找到 30 个CSV文件
找到 48 个CSV文件


## 4QAD

In [ ]:
warnings.filterwarnings('ignore')

def extract_antibiotic_code(drug_item_code):
    if pd.isna(drug_item_code):
        return None

    code_str = str(drug_item_code).strip()
    match = re.match(r'([A-Za-z]+)', code_str)
    if match:
        return match.group(1).upper()
    else:
        return code_str.upper()

def is_vancomycin(antibiotic_code):
    if pd.isna(antibiotic_code):
        return False
    return antibiotic_code.startswith('VANC')

def parse_frequency(freq_str, start_date, end_date):
    if pd.isna(freq_str) or pd.isna(start_date) or pd.isna(end_date):
        return []

    freq = str(freq_str).upper().strip()

    if "EVERY" in freq and "HOURS" in freq:
        numbers = re.findall(r'\d+', freq)
        if numbers:
            interval_hours = int(numbers[0])

            dates = []
            current_time = datetime.combine(start_date, datetime.min.time())
            end_time = datetime.combine(end_date, datetime.min.time()) + timedelta(days=1)

            while current_time <= end_time:
                dates.append(current_time.date())
                current_time += timedelta(hours=interval_hours)

            unique_dates = sorted(set(dates))
            return [d for d in unique_dates if start_date <= d <= end_date]

    dates = []
    current_date = start_date
    while current_date <= end_date:
        dates.append(current_date)
        current_date += timedelta(days=1)

    return dates

def load_antibiotics_data(antibiotic_path):
    print("加载抗生素数据...")

    antibiotic_files = glob.glob(os.path.join(antibiotic_path, "*.csv"))
    if not antibiotic_files:
        print(f"错误: 在路径 {antibiotic_path} 下未找到CSV文件")
        return None

    print(f"找到 {len(antibiotic_files)} 个抗生素数据文件")
    all_dfs = []

    for i, file in enumerate(antibiotic_files):
        try:
            df = pd.read_csv(file, low_memory=False)
            required_cols = ['HN Number', 'Prescription Start Date', 'Prescription End Date',
                           'Drug Item Code', 'Route', 'Drug Frequency']
            
            optional_cols = []
            if 'Dispensing Date (yyyy-mm-dd)' in df.columns:
                optional_cols.append('Dispensing Date (yyyy-mm-dd)')
            if 'Dispensing Duration' in df.columns:
                optional_cols.append('Dispensing Duration')
            
            all_cols = required_cols + optional_cols
            df = df[all_cols].copy()

            if 'Dispensing Date (yyyy-mm-dd)' in df.columns:
                df['Dispensing Date (yyyy-mm-dd)'] = pd.to_datetime(df['Dispensing Date (yyyy-mm-dd)'], errors='coerce').dt.date

            df['Prescription Start Date'] = pd.to_datetime(df['Prescription Start Date'], errors='coerce').dt.date
            df['Prescription End Date'] = pd.to_datetime(df['Prescription End Date'], errors='coerce').dt.date

            missing_start_mask = df['Prescription Start Date'].isna()
            if 'Dispensing Date (yyyy-mm-dd)' in df.columns:
                df.loc[missing_start_mask, 'Prescription Start Date'] = df.loc[missing_start_mask, 'Dispensing Date (yyyy-mm-dd)']

            if 'Dispensing Duration' in df.columns and 'Prescription Start Date' in df.columns:
                try:
                    df['Dispensing Duration'] = pd.to_numeric(df['Dispensing Duration'], errors='coerce')
                    for idx, row in df.iterrows():
                        if pd.isna(row['Prescription End Date']) and not pd.isna(row['Prescription Start Date']) and not pd.isna(row['Dispensing Duration']):
                            duration_days = int(row['Dispensing Duration'])
                            if duration_days > 0:
                                end_date = row['Prescription Start Date'] + timedelta(days=duration_days)
                                df.at[idx, 'Prescription End Date'] = end_date
                except Exception as e:
                    print(f"  转换Dispensing Duration时出错: {str(e)}")

            df['Antibiotic_Code'] = df['Drug Item Code'].apply(extract_antibiotic_code)
            df['Is_Vancomycin'] = df['Antibiotic_Code'].apply(is_vancomycin)
            all_dfs.append(df)

        except Exception as e:
            print(f"处理文件 {os.path.basename(file)} 时出错: {str(e)}")
            continue

    if not all_dfs:
        print("错误: 没有成功加载任何抗生素数据")
        return None

    antibiotics_df = pd.concat(all_dfs, ignore_index=True)

    print("数据清洗...")
    original_count = len(antibiotics_df)
    antibiotics_df = antibiotics_df.dropna(subset=['Prescription Start Date', 'Prescription End Date'])
    removed_count = original_count - len(antibiotics_df)
    if removed_count > 0:
        print(f"移除了 {removed_count} 行日期为空的记录")

    original_count = len(antibiotics_df)
    antibiotics_df = antibiotics_df.dropna(subset=['Antibiotic_Code'])
    removed_count = original_count - len(antibiotics_df)
    if removed_count > 0:
        print(f"移除了 {removed_count} 行抗生素代码为空的记录")

    valid_routes = ['PARENTERAL', 'ORAL', 'INTRAVENOUS']
    original_count = len(antibiotics_df)
    antibiotics_df = antibiotics_df[antibiotics_df['Route'].isin(valid_routes)]
    removed_count = original_count - len(antibiotics_df)
    if removed_count > 0:
        print(f"移除了 {removed_count} 行给药途径不在{valid_routes}范围内的记录")

    original_count = len(antibiotics_df)
    antibiotics_df = antibiotics_df.dropna(subset=['Route'])
    removed_count = original_count - len(antibiotics_df)
    if removed_count > 0:
        print(f"移除了 {removed_count} 行Route为空的记录")

    invalid_dates_mask = antibiotics_df['Prescription End Date'] < antibiotics_df['Prescription Start Date']
    if invalid_dates_mask.any():
        invalid_count = invalid_dates_mask.sum()
        antibiotics_df.loc[invalid_dates_mask, ['Prescription Start Date', 'Prescription End Date']] = antibiotics_df.loc[invalid_dates_mask, ['Prescription End Date', 'Prescription Start Date']].values

    print(f"抗生素数据处理完成，共 {len(antibiotics_df)} 行记录")
    return antibiotics_df

def load_window_data(window_path, year):
    print("加载窗口期数据...")

    window_files = glob.glob(os.path.join(window_path, f"*{year}*.csv"))
    if not window_files:
        print(f"错误: 在路径 {window_path} 下未找到CSV文件")
        return None

    print(f"找到 {len(window_files)} 个窗口期数据文件")
    all_dfs = []

    for i, file in enumerate(window_files):
        try:
            df = pd.read_csv(file, low_memory=False)
            all_dfs.append(df)
        except Exception as e:
            print(f"处理文件 {os.path.basename(file)} 时出错: {str(e)}")
            continue

    if not all_dfs:
        print("错误: 没有成功加载任何窗口期数据")
        return None

    window_df = pd.concat(all_dfs, ignore_index=True)
    print(f"窗口期数据加载完成，共 {len(window_df)} 行记录")

    date_cols = ['blood culture date', 'window_period_start', 'window_period_end']
    for col in date_cols:
        if col in window_df.columns:
            window_df[col] = pd.to_datetime(window_df[col], errors='coerce').dt.date
        else:
            print(f"警告: 列 {col} 不存在")

    original_count = len(window_df)
    window_df = window_df.dropna(subset=['blood culture date', 'window_period_start', 'window_period_end', 'HN Number'])
    removed_count = original_count - len(window_df)
    if removed_count > 0:
        print(f"移除了 {removed_count} 行关键信息为空的记录")

    print(f"窗口期数据处理完成，共 {len(window_df)} 行记录")
    return window_df

def create_patient_medication_calendar(antibiotics_df):
    print("创建病人给药日历...")
    patient_calendars = {}
    groups = list(antibiotics_df.groupby('HN Number'))
    total_patients = len(groups)
    print(f"总共 {total_patients} 条住院记录需要处理")

    for i, (hn, group) in enumerate(groups):
        if (i+1) % 500 == 0 or i == 0:
            print(f"处理住院记录 {i+1}/{total_patients}")

        calendar = {}
        for _, row in group.iterrows():
            start_date = row['Prescription Start Date']
            end_date = row['Prescription End Date']
            antibiotic_code = row['Antibiotic_Code']
            route = row['Route']
            frequency = row['Drug Frequency']
            is_vancomycin = row['Is_Vancomycin']

            if end_date < start_date:
                start_date, end_date = end_date, start_date

            try:
                dose_dates = parse_frequency(frequency, start_date, end_date)
                for dose_date in dose_dates:
                    if dose_date not in calendar:
                        calendar[dose_date] = {
                            'drugs': set(),
                            'routes': set(),
                            'drug_info': {}
                        }

                    calendar[dose_date]['drugs'].add(antibiotic_code)
                    calendar[dose_date]['routes'].add(route)

                    if antibiotic_code not in calendar[dose_date]['drug_info']:
                        calendar[dose_date]['drug_info'][antibiotic_code] = {
                            'route': route,
                            'is_vancomycin': is_vancomycin,
                            'first_use_date': dose_date
                        }
            except Exception as e:
                continue

        patient_calendars[hn] = calendar

    print(f"给药日历创建完成，共 {len(patient_calendars)} 条住院记录")
    return patient_calendars

def is_new_antibiotic(antibiotic_code, route, dose_date, patient_calendar, is_vancomycin):
    two_days_ago = dose_date - timedelta(days=2)

    for check_date, med_info in patient_calendar.items():
        if two_days_ago <= check_date < dose_date:
            if antibiotic_code in med_info['drugs']:
                if is_vancomycin:
                    if antibiotic_code in med_info['drug_info']:
                        prev_route = med_info['drug_info'][antibiotic_code]['route']
                        if route == prev_route:
                            return False
                        else:
                            continue
                    else:
                        return False
                else:
                    return False
    return True

def check_4qad_for_window(hn, window_row, patient_calendar):
    window_start = window_row['window_period_start']
    window_end = window_row['window_period_end']

    window_meds = {}
    for date, med_info in patient_calendar.items():
        if window_start <= date <= window_end:
            window_meds[date] = med_info

    if not window_meds:
        return False

    sorted_dates = sorted(window_meds.keys())
    first_new_antibiotic_date = None

    for date in sorted_dates:
        med_info = window_meds[date]
        for antibiotic_code in med_info['drugs']:
            if antibiotic_code in med_info['drug_info']:
                drug_info = med_info['drug_info'][antibiotic_code]
                route = drug_info['route']
                is_vancomycin = drug_info['is_vancomycin']

                if is_new_antibiotic(antibiotic_code, route, date, patient_calendar, is_vancomycin):
                    first_new_antibiotic_date = date
                    break
        if first_new_antibiotic_date:
            break

    if not first_new_antibiotic_date:
        return False

    qad_dates = []
    current_date = first_new_antibiotic_date
    max_gap = 1
    gaps_used = 0
    max_days_to_check = 366

    for _ in range(max_days_to_check):
        if len(qad_dates) >= 4:
            break

        if current_date in patient_calendar:
            qad_dates.append(current_date)
            gaps_used = 0
        else:
            if gaps_used < max_gap:
                gaps_used += 1
            else:
                break
        current_date += timedelta(days=1)

    if len(qad_dates) < 4:
        return False

    has_parenteral = False
    for qad_date in qad_dates[:4]:
        if qad_date in patient_calendar:
            routes = patient_calendar[qad_date]['routes']
            if 'PARENTERAL' in routes or 'INTRAVENOUS' in routes:
                has_parenteral = True
                break

    if not has_parenteral:
        return False

    return True

def add_clusters_column(df):
    cluster_mapping = {}

    hkec_hospitals = ['CHC', 'PYN', 'RH', 'SJH', 'TWE', 'WCH']
    for hospital in hkec_hospitals:
        cluster_mapping[hospital] = 'HKEC'

    hkwc_hospitals = ['DKC', 'FYK', 'GH', 'ML', 'NLH', 'QMH', 'TWH', 'TYH']
    for hospital in hkwc_hospitals:
        cluster_mapping[hospital] = 'HKWC'

    kcc_hospitals = ['BH', 'HCH', 'HKE', 'KH', 'KWH', 'OLM', 'QEH', 'RC', 'WTS']
    for hospital in kcc_hospitals:
        cluster_mapping[hospital] = 'KCC'

    kec_hospitals = ['HHH', 'MT', 'TKO', 'UCH']
    for hospital in kec_hospitals:
        cluster_mapping[hospital] = 'KEC'

    kwc_hospitals = ['CMC', 'KCH', 'LCK', 'NLT', 'PMH', 'YCH']
    for hospital in kwc_hospitals:
        cluster_mapping[hospital] = 'KWC'

    ntec_hospitals = ['AHN', 'BBH', 'CHS', 'NDH', 'PWH', 'SH', 'TPH']
    for hospital in ntec_hospitals:
        cluster_mapping[hospital] = 'NTEC'

    ntwc_hospitals = ['CPH', 'POH', 'SLH', 'TMH', 'TSH']
    for hospital in ntwc_hospitals:
        cluster_mapping[hospital] = 'NTWC'

    df['Clusters'] = df['Institution (IPAS)'].map(cluster_mapping)

    if df['Clusters'].isna().any():
        df['Clusters'] = df['Clusters'].fillna('UNKNOWN')

    return df

def main():
    print("=== 4 QAD Filter 程序开始 ===")

    # Fixed: Added year parameter to loop through years
    for year in range(2001, 2024):
        print(f"\n处理年份: {year}")
        
        antibiotic_path = f"./presumed_infection/2.Data/Microbial/{year}/"
        window_path = f"./presumed_infection/2.Data/Window_Period/"
        output_path_csv = f"./presumed_infection/2.Data/Output/Presumedinfection_csv/"
        output_path_txt = f"./presumed_infection/2.Data/Output/Presumedinfection_txt/"

        # Create output directories if they don't exist
        for path in [output_path_csv, output_path_txt]:
            if not os.path.exists(path):
                os.makedirs(path, exist_ok=True)

        # Load data
        antibiotics_df = load_antibiotics_data(antibiotic_path)
        if antibiotics_df is None or len(antibiotics_df) == 0:
            print(f"年份 {year}: 无法加载抗生素数据或数据为空，跳过")
            continue

        window_df = load_window_data(window_path, year)
        if window_df is None or len(window_df) == 0:
            print(f"年份 {year}: 无法加载窗口期数据或数据为空，跳过")
            continue

        # Create medication calendar
        patient_calendars = create_patient_medication_calendar(antibiotics_df)

        # Check 4QAD conditions
        met_4qad_indices = []
        total_windows = len(window_df)
        print("检查4QAD条件...")
        
        for idx, window_row in window_df.iterrows():
            hn = window_row['HN Number']
            if hn not in patient_calendars:
                continue

            if check_4qad_for_window(hn, window_row, patient_calendars[hn]):
                met_4qad_indices.append(idx)

            if (idx + 1) % 1000 == 0:
                print(f"已处理 {idx + 1}/{total_windows} 个血培养窗口")

        # Extract results
        if met_4qad_indices:
            results_df = window_df.iloc[met_4qad_indices].copy()
            print(f"找到 {len(results_df)} 个满足4QAD条件的记录")
        else:
            print("没有找到满足4QAD条件的记录")
            continue

        # Add Clusters column
        results_df = add_clusters_column(results_df)

        # Select and deduplicate
        selected_columns = ['Reference Key', 'Institution (IPAS)', 'HN Number', 'Clusters']
        if all(col in results_df.columns for col in selected_columns):
            final_df = results_df[selected_columns].copy()
            
            original_count = len(final_df)
            final_df = final_df.drop_duplicates(subset=['HN Number'], keep='first')
            print(f"去重后保留 {len(final_df)} 条记录 (去除了 {original_count - len(final_df)} 条重复记录)")

            # Save results
            csv_file = os.path.join(output_path_csv, f"Presumed_Infection_List_{year}.csv")
            final_df.to_csv(csv_file, index=False)
            print(f"CSV文件已保存: {csv_file}")

            txt_file = os.path.join(output_path_txt, f"Presumed_Infection_List_{year}.txt")
            txt_columns = ['Reference Key', 'Institution (IPAS)', 'HN Number']
            if all(col in final_df.columns for col in txt_columns):
                txt_df = final_df[txt_columns].copy()
                txt_df.to_csv(txt_file, sep='\t', index=False)
                print(f"TXT文件已保存: {txt_file}")

            # Statistics
            print(f"\n=== 统计信息 ===")
            print(f"总血培养窗口数: {total_windows}")
            print(f"满足4QAD条件: {len(results_df)} ({len(results_df)/total_windows*100:.1f}%)")
            print(f"去重后唯一病人数: {len(final_df)}")

            if 'Clusters' in final_df.columns and len(final_df) > 0:
                print("\nCluster分布:")
                cluster_counts = final_df['Clusters'].value_counts()
                total_patients = len(final_df)
                for cluster, count in cluster_counts.items():
                    print(f"  {cluster}: {count} 个病人 ({count/total_patients*100:.1f}%)")
        else:
            print("错误: 数据中缺少必要的列")

    print("\n=== 程序执行完成 ===")
    return None

if __name__ == "__main__":
    try:
        results_df = main()
    except Exception as e:
        print(f"程序执行出错: {str(e)}")
        import traceback
        traceback.print_exc()

=== 4 QAD Filter 程序开始 ===

处理年份: 2001
加载抗生素数据...
找到 22 个抗生素数据文件
数据清洗...
移除了 379236 行日期为空的记录
移除了 242127 行给药途径不在['PARENTERAL', 'ORAL', 'INTRAVENOUS']范围内的记录
抗生素数据处理完成，共 171630 行记录
加载窗口期数据...
找到 1 个窗口期数据文件
窗口期数据加载完成，共 88976 行记录
窗口期数据处理完成，共 88976 行记录
创建病人给药日历...
总共 33410 条住院记录需要处理
处理住院记录 1/33410
处理住院记录 500/33410
处理住院记录 1000/33410
处理住院记录 1500/33410
处理住院记录 2000/33410
处理住院记录 2500/33410
处理住院记录 3000/33410
处理住院记录 3500/33410
处理住院记录 4000/33410
处理住院记录 4500/33410
处理住院记录 5000/33410
处理住院记录 5500/33410
处理住院记录 6000/33410
处理住院记录 6500/33410
处理住院记录 7000/33410
处理住院记录 7500/33410
处理住院记录 8000/33410
处理住院记录 8500/33410
处理住院记录 9000/33410
处理住院记录 9500/33410
处理住院记录 10000/33410
处理住院记录 10500/33410
处理住院记录 11000/33410
处理住院记录 11500/33410
处理住院记录 12000/33410
处理住院记录 12500/33410
处理住院记录 13000/33410
处理住院记录 13500/33410
处理住院记录 14000/33410
处理住院记录 14500/33410
处理住院记录 15000/33410
处理住院记录 15500/33410
处理住院记录 16000/33410
处理住院记录 16500/33410
处理住院记录 17000/33410
处理住院记录 17500/33410
处理住院记录 18000/33410
处理住院记录 18500/33410
处理住院记录 19000/33410
处理住院记录